In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore") 

### Data Collection and Analysis

PIMA Diabetes Dataset

In [2]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.shape

(768, 9)

In [4]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
df['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [6]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [7]:
df.groupby("Outcome").mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [8]:
columns = list(df.columns)
columns = columns[:-1]
columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [9]:
for column in columns:
    upper_limit = df[column].mean() + 3*df[column].std()
    lower_limit = df[column].mean() - 3*df[column].std()
    df =  df[(df[column]<upper_limit) & (df[column]>lower_limit)]
df.shape

(685, 9)

In [10]:
# separating the data and labels
X = df.drop(columns = 'Outcome', axis=1)
y = df['Outcome']

In [11]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X, y = smote.fit_resample(X, y)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, stratify=y, random_state=2)

In [13]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [14]:
classifier = svm.SVC(kernel='linear')

classifier.fit(X_train, y_train)

SVC(kernel='linear')

### Model evaluation

In [15]:
# Model evaluation on train data
train_pred = classifier.predict(X_train)
print('Accuracy score on train data: ', accuracy_score(train_pred, y_train))

Accuracy score on train data:  0.7479564032697548


In [16]:
# Model evaluation on test data
test_pred = classifier.predict(X_test)
print('Accuracy score on test data: ', accuracy_score(test_pred, y_test))

Accuracy score on test data:  0.75


## Using pipeline

In [17]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('scaler', StandardScaler()), ('svc', svm.SVC())])

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])

## Saving the model

In [18]:
import pickle
filename = 'trained.pkl'
pickle.dump(classifier, open(filename, 'wb'))

In [19]:
load_model = pickle.load(open(filename, 'rb'))

'Accuracy on test data:'+ str(accuracy_score(load_model.predict(X_test),y_test))

'Accuracy on test data:0.75'

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [21]:
model = tf.keras.Sequential([
    layers.Dense(units = 16, activation = 'relu', input_shape = [8]),
    layers.Dropout(rate=0.3),
    layers.Dense(units=1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50)

Epoch 1/50
23/23 [==============================] - 4s 3ms/step - loss: 0.6850 - accuracy: 0.6226
Epoch 2/50
23/23 [==============================] - 0s 4ms/step - loss: 0.6345 - accuracy: 0.6662
Epoch 3/50
23/23 [==============================] - 0s 4ms/step - loss: 0.6331 - accuracy: 0.6403
Epoch 4/50
23/23 [==============================] - 0s 4ms/step - loss: 0.6141 - accuracy: 0.6771
Epoch 5/50
23/23 [==============================] - 0s 4ms/step - loss: 0.6065 - accuracy: 0.6662
Epoch 6/50
23/23 [==============================] - 0s 4ms/step - loss: 0.5790 - accuracy: 0.7221
Epoch 7/50
23/23 [==============================] - 0s 4ms/step - loss: 0.5693 - accuracy: 0.7207
Epoch 8/50
23/23 [==============================] - 0s 3ms/step - loss: 0.5579 - accuracy: 0.7207
Epoch 9/50
23/23 [==============================] - 0s 4ms/step - loss: 0.5655 - accuracy: 0.7125
Epoch 10/50
23/23 [==============================] - 0s 4ms/step - loss: 0.5381 - accuracy: 0.7289
Epoch 11/50
23/23 [

In [22]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

6/6 [==============================] - 0s 2ms/step - loss: 0.4566 - accuracy: 0.7609
Accuracy: 76.09


In [23]:
model.save('my_model.keras')

In [24]:
y_preds = model.predict(X_test)
y_preds = np.round(y_preds)
y_preds

6/6 [==============================] - 0s 1ms/step


array([[0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],

In [25]:
from sklearn.metrics import classification_report
print("Classification Report: \n", classification_report(y_test, y_preds))

Classification Report: 
               precision    recall  f1-score   support

           0       0.75      0.78      0.77        92
           1       0.77      0.74      0.76        92

    accuracy                           0.76       184
   macro avg       0.76      0.76      0.76       184
weighted avg       0.76      0.76      0.76       184

